In [1]:
import glob
import pandas as pd
import os
import librosa
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [8]:
count = 2000
df = pd.read_csv("correct_data.csv")
root = "../frames/"
files = df.file.values[:count]
files = df.file.values[:count]
targets = df.target.values[:count]

In [9]:
files = ["Audio_files/laugh_1.wav"]
targets = ["0"*201]

In [19]:
def write_tensor_files_mod(files, targets):
    save_folder = "../data/"
    for i, file in enumerate(files):
        pref = "../"
        file_path = pref + file
        aud, sr = librosa.load(file_path, sr = None)
        if aud.shape[0] > 320_000:
            aud = aud[:320_000]
        win_len = int(sr * 0.2)
        hop = int(sr * 0.05)
        aud_idx = 0
        target = targets[i]
        for j in range(len(target)-1):
            temp_len = len(aud[aud_idx:aud_idx + win_len])
            
            
            if temp_len < 6400:
                pad_len = 6400 - temp_len
                temp_aud = np.pad(aud, pad_len, mode = "reflect")[pad_len:]
                torch.save(torch.tensor(temp_aud[aud_idx:aud_idx + win_len], dtype = torch.float32), 
                           root + file + str(np.random.randint(0, 10000)) + str(j) + ".pt")
                
            else:
                torch.save(torch.tensor(aud[aud_idx:aud_idx + win_len], dtype = torch.float32), 
                           root + file[:-4] + str(np.random.randint(0, 10000)) + str(j) + ".pt")
            aud_idx += hop    
                
write_tensor_files_mod(files, targets)       

In [224]:
split_ = lambda x: x.split("/")[-1]

def find_tensor_files(root):
    return list(map(split_, glob.glob(root + "*")))

all_tensor_files = find_tensor_files(root)
len(all_tensor_files)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


296537

In [225]:
all_tensor_files[5]


'9fDTV2zd4Yo_30.wav&*&132.pt'

In [89]:
def num_frames(file):
    path = 'data_32/' + file
    aud, sr = librosa.load(path, sr = None)
    return aud.shape[0] / (0.05 * sr)

In [90]:
df['num_frames'] = df['file'].apply(num_frames)

In [193]:
df.head(17)

,file,category,target
0,4GV3Cd2upt8_30.wav,printer,0000000000000000000000000000000000000000000000...
1,6fSCG3bLEio_70.wav,typing,0000000000000000000000000000000000000000000000...
2,JP2sPYB_Bhs_30.wav,printer,0000000000000000000000000000000000000000000000...
3,8jNYEw7ztUE_90.wav,laughter,1111111000000000000000000000000000000000000000...
4,E2bFAoGuCBU_23.wav,white,0000000000000000000000000000000000000000000000...
5,KGJK1UO-F4k_230.wav,conservation,0000000000000000000000000000000000000000000000...
6,6IFr_7q3NZo_150.wav,laughter,0000000000001111111111111100000000000000000000...
7,I8tnpoJSDRE_30.wav,white,0000000000000000000000000000000000000000000000...
8,Mt5wTIMQMGE_300.wav,white,0000000000000000000000000000000000000000000000...
9,2YGDu50akBc_10.wav,laughter,0000000000000000000000000000000000000000000000...


In [93]:
df[df['num_frames'] != df['target_len']-1].shape

(983, 5)

In [197]:
df['target_len'] = df["target"].apply(len)

In [198]:
df.head(3)

,file,category,target,target_len
0,4GV3Cd2upt8_30.wav,printer,0000000000000000000000000000000000000000000000...,201
1,6fSCG3bLEio_70.wav,typing,0000000000000000000000000000000000000000000000...,201
2,JP2sPYB_Bhs_30.wav,printer,0000000000000000000000000000000000000000000000...,201


In [212]:
df.head(100)[df['target_len'] < 201]




/tmp/ipykernel_1311333/1792783503.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.head(100)[df['target_len'] < 201]


,file,category,target,target_len
4,E2bFAoGuCBU_23.wav,white,0000000000000000000000000000000000000000000000...,181
11,4AUtgC1dPwA_29.wav,white,0000000000000000000000000000000000000000000000...,189
20,Ba8fS5iv1_Y_20.wav,conservation,0000000000000000000000000000000000000000000000...,193
30,MVjJAGC0UAY_3.wav,printer,0000000000000000000000000000000000000000000000...,195
53,MG3g4sfSxXo_0.wav,giggle,0000000000000000000000000000000000000000000000...,200
81,7TXDyi7BacU_0.wav,white,0000000000000000000000000000000000000000000000...,200
83,AM0IH1SUJa8_0.wav,laughter,0000000000000000000000000000000000000000000000...,82
91,LZqGPfZIojo_7.wav,printer,0000000000000000000000000000000000000000000000...,195
93,KdPKmF_in8s_14.wav,white,0000000000000000000000000000000000000000000000...,181
94,Fub2Ll-Xfec_12.wav,white,0000000000000000000000000000000000000000000000...,189


In [5]:
# Проверим заканчивается ли название файла символами --
all_files = df.file

isend = lambda x: x[:-4].endswith("--")
all_files.map(isend).sum()

0

In [139]:
all_tensor_files[11]

'2YGDu50akBc_10.wav&*&182.pt'

In [171]:
files[9]

'2YGDu50akBc_10.wav'

In [216]:
ex_file = "KvjUbJ2YQOo_13.wav"
res = []
for file in all_tensor_files:
    
    if file.startswith(ex_file[:-4]):
        res.append(int(file.split("&*&")[-1][:-3]))

In [217]:
max(res)

180

In [218]:
df[df['file'] == ex_file]

,file,category,target,target_len
95,KvjUbJ2YQOo_13.wav,snicker,0000000000000000000111111111111111111111111111...,182


In [221]:
df[df['file'] == ex_file]['target'].values

array(['00000000000000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000000000000000000000000000000000000000000000000000000000000000000'],
      dtype=object)